In [80]:
import json
import os
import re

In [81]:
dir = "../benchmark/hindi_agro"
fixed_dir = "../benchmark/hindi_agro_fixed"
def dn(file, _dir=dir):
    return os.path.join(_dir, file)
q=json.load(open(dn("300_queries_hindi_english.json")))

In [140]:
def fix_question_text(text):
    # Remove <question> and </question> tags
    text = text.replace('<question>', '').replace('</question>', '')
    # Replace "Bhai, " with empty string
    text = text.replace('Bhai, ', '')
    return text.strip()


with open(dn("english_questions.jsonl", fixed_dir), "w") as out:
    for i, (k, v) in enumerate(q['english_question'].items()):
        out.write(json.dumps({'text': fix_question_text(v), 'id': str(i), 'relevant': str(q['chunk_id'][k])},
                             ensure_ascii=False) + "\n")


In [ ]:
with open(dn("hindi_questions.jsonl"), "w") as out:
    for i, (k, v) in enumerate(q['hindi_question'].items()):
        out.write(json.dumps({'text': v, 'id': str(i), 'relevant': str(q['chunk_id'][k])}, ensure_ascii=False)+"\n")

In [132]:
data = json.load(open(dn("cleaned_agri_docs_translation.json")))
vals = list(data['page_content'].values())

In [ ]:
data['page_content']

In [85]:
# The pattern we're looking for: \u6 followed by exactly 3 digits
pattern = r'\\u6\d{3}'
def find_unicode_pattern(input_string):
    """
    Find all instances of the pattern \\u6xxx in a string, where xxx represents three digits.

    Args:
        input_string (str): The string to search in

    Returns:
        list: A list of all matching patterns found in the string
    """

    # Find all matches
    matches = re.findall(pattern, json.dumps(input_string, ensure_ascii=True))

    return matches

def find_bad_chars(input_string):
    ret = ""
    for c in input_string:
        ordc = ord(c)
        if (0x6000 <= ordc <= 0x6FFF) and (ordc & 0xff == 0x00) and c not in ret:
            ret += c
    return ret

In [86]:
def char_to_unicode_escape(char):
    """
    Convert a single UTF-8 character to its \\u Unicode escape sequence.

    Args:
        char (str): A single Unicode character

    Returns:
        str: The \\u Unicode escape sequence for the character
    """
    # Get the Unicode code point (as an integer)
    code_point = ord(char)

    # Format it as \u escape sequence
    if code_point <= 0xFFFF:
        # Basic Multilingual Plane (BMP) - use \u notation
        return f"\\u{code_point:04x}"
    else:
        # Supplementary Planes - use \U notation (8 digits)
        return f"\\U{code_point:08x}"

def fix_char(char):
    code_point = ord(char)
    if (0x6000 <= code_point <= 0x6FFF) and (code_point & 0xff == 0x00):
        return chr(int(code_point/0x100))
    else:
        return char


In [87]:
# Check to see which chars are bad
val = re.compile(r'(\\u6\d\d\d)')

fix = {}
for k, v in data['page_content'].items():
    matches = find_bad_chars(v)
    if matches:
        for m in matches:
            fix[m] = fix_char(m)

for k, v in data['page_content'].items():
    matches = find_bad_chars(v)
    if matches:
        r = v
        for m in matches:
            r = r.replace(m, fix[m])
        data['page_content'][k] = r
    v = v.replace("\t", " ")

In [135]:
# Check to see which chars are bad
def fix_corpus(corpus_label):
    val = re.compile(r'(\\u6\d\d\d)')

    fix = {}
    for k, v in data[corpus_label].items():
        matches = find_bad_chars(v)
        if matches:
            for m in matches:
                fix[m] = fix_char(m)
    print(f"Fixing {corpus_label}, with the replacements {fix}")
    for k, v in data[corpus_label].items():
        matches = find_bad_chars(v)
        r = v
        if matches:
            for m in matches:
                r = r.replace(m, fix[m])
        r = r.replace("\t", " ")
        r = re.sub(r" +", " ", r)
        data[corpus_label][k] = r


In [104]:
chunk_id=data['chunk_id']
chunk_id['252']
data['page_content']['252']

'Chapter: (b) Diseases \n1. Seed  rot  and  seedling  blight ( Fusarium,  Penicillium,  Aspergillus spp. ):  Poor germination,\t unthrifty\t seedlings\t and\t seedling\t mortality\t are\t the\t symptoms. Use disease free seed. 2. Post-flowering stalk rots ( Macrophomina phaseolina, Cephalosporium maydis, Fusarium spp. ) :\tAll\tthe\tstalk\trots\tresult\tin\twilting\tof\tplants\tafter\tflowering\tfollowed by\tweakening\tof\tstem. Infection\twith\tM.\tphaseolina\tresult\tin\tblack\tsporulation\tin the  pith  region. Dry  conditions  at  tasseling  stage  favour  disease  development. Water\tstress\tat\tthe\ttime\tof\tflowering\tfavours\tthe\tdisease. Grow\t\trecommended varieties. Note : No incidence has been observed of most of the diseases prevalent during kharif season,\tnamely\tmaydis\tleaf\tblight,\tbanded\tleaf\tand\tsheath\tblight,\tbrown\tstripe\tdowny mildew\tand\tpre-flowering\tstalk\trots.'

In [131]:
data['page_content']['252']

'Chapter: (b) Diseases \n1. Seed  rot  and  seedling  blight ( Fusarium,  Penicillium,  Aspergillus spp. ):  Poor germination,  unthrifty  seedlings  and  seedling  mortality  are  the  symptoms. Use disease free seed. 2. Post-flowering stalk rots ( Macrophomina phaseolina, Cephalosporium maydis, Fusarium spp. ) : All the stalk rots result in wilting of plants after flowering followed by weakening of stem. Infection with M. phaseolina result in black sporulation in the  pith  region. Dry  conditions  at  tasseling  stage  favour  disease  development. Water stress at the time of flowering favours the disease. Grow  recommended varieties. Note : No incidence has been observed of most of the diseases prevalent during kharif season, namely maydis leaf blight, banded leaf and sheath blight, brown stripe downy mildew and pre-flowering stalk rots.'

In [122]:
for c in vals[23]:
    if ord(c)>256:
        print(f"{c}: {char_to_unicode_escape(c)} - fix={fix_char(c)}")

•: \u2022 - fix=•
•: \u2022 - fix=•


In [136]:
fix_corpus("page_content")
with open(dn("english_corpus.jsonl", fixed_dir), "w") as out:
    for k, v in data['page_content'].items() :
        out.write(json.dumps({'text': v, 'id': str(data['chunk_id'][k])}, ensure_ascii=False)+"\n")

Fixing page_content, with the replacements {'昀': 'f', '椀': 'i', '氀': 'l'}


In [100]:
[k for k, d in data['chunk_id'].items() if d==4700852927978117981]

['252']

In [113]:
fix_corpus('Hindi_page_content')

Fixing Hindi_page_content, with the replacements {'昀': 'f', '椀': 'i'}


In [114]:
with open(dn("hindi_corpus.jsonl", fixed_dir), "w") as out:
    for k, v in data['Hindi_page_content'].items() :
        out.write(json.dumps({'text': v, 'id': str(data['chunk_id'][k])}, ensure_ascii=False)+"\n")